## TP Final - Optimizacion

In [10]:
import cplex

In [37]:
TOLERANCE =10e-6 

class Orden:
    def __init__(self):
        self.id = 0
        self.beneficio = 0
        self.trabajadores_necesarios = 0
    
    def load(self, row):
        self.id = int(row[0])
        self.beneficio = int(row[1])
        self.trabajadores_necesarios = int(row[2])
        

class FieldWorkAssignment:
    def __init__(self):
        self.cantidad_trabajadores = 0
        self.cantidad_ordenes = 0
        self.ordenes = []
        self.conflictos_trabajadores = []
        self.ordenes_correlativas = []
        self.ordenes_conflictivas = []
        self.ordenes_repetitivas = []
        self.diccionario_indices = {}
        self.variables_totales=0
        self.tramos_salario=4
        self.valor_tramo=[-1_000, -1_200,-1_400,-1_500 ]
        self.dias_semana=6
        self.turnos=5
        

    def load(self,filename):
        # Abrimos el archivo.
        f = open(filename)

        # Leemos la cantidad de trabajadores
        self.cantidad_trabajadores = int(f.readline())
        
        # Leemos la cantidad de ordenes
        self.cantidad_ordenes = int(f.readline())
        
        # Leemos cada una de las ordenes.
        self.ordenes = []
        for i in range(self.cantidad_ordenes):
            row = f.readline().split(' ')
            orden = Orden()
            orden.load(row)
            self.ordenes.append(orden)

        # Leemos la cantidad de conflictos entre los trabajadores
        cantidad_conflictos_trabajadores = int(f.readline())
        
        # Leemos los conflictos entre los trabajadores
        self.conflictos_trabajadores = []
        for i in range(cantidad_conflictos_trabajadores):
            row = f.readline().split(' ')
            self.conflictos_trabajadores.append(list(map(int,row)))
            
        # Leemos la cantidad de ordenes correlativas
        cantidad_ordenes_correlativas = int(f.readline())
        
        # Leemos las ordenes correlativas
        self.ordenes_correlativas = []
        for i in range(cantidad_ordenes_correlativas):
            row = f.readline().split(' ')
            self.ordenes_correlativas.append(list(map(int,row)))
            
        # Leemos la cantidad de ordenes conflictivas
        cantidad_ordenes_conflictivas = int(f.readline())
        
        # Leemos las ordenes conflictivas
        self.ordenes_conflictivas = []
        for i in range(cantidad_ordenes_conflictivas):
            row = f.readline().split(' ')
            self.ordenes_conflictivas.append(list(map(int,row)))
        
        
        # Leemos la cantidad de ordenes repetitivas
        cantidad_ordenes_repetitivas = int(f.readline())
        
        # Leemos las ordenes repetitivas
        self.ordenes_repetitivas = []
        for i in range(cantidad_ordenes_repetitivas):
            row = f.readline().split(' ')
            self.ordenes_repetitivas.append(list(map(int,row)))


        
        I=self.cantidad_trabajadores  #trabajador
        J=self.cantidad_ordenes  #orden o tarea
        K=self.dias_semana  #dia de semana
        L=self.turnos  #turno
        N=self.tramos_salario  #tramos
        W=N-1  #auxiliar activa tramos
        for i in range(I):  # i 
            for j in range(J):  # j 
                for k in range(K):  # k 
                    for l in range(L):  # l 
                        clave = f"X_{i}_{j}_{k}_{l}"  
                        valor = i * L*K*J + j * L*K + k * L + l  # Valor correlativo
                        self.diccionario_indices[clave] = valor

        max_valor = max(self.diccionario_indices.values())
        for i in range(I):  # i 
            for k in range(K):  # k 
                clave = f"D_{i}_{k}"
                valor = max_valor+1+i*K+k
                self.diccionario_indices[clave] = valor

        max_valor = max(self.diccionario_indices.values())
        for j in range(J):  # j 
            for k in range(K):  # k 
                for l in range(L):  # l 
                    clave = f"R_{j}_{k}_{l}"  
                    valor = max_valor+1+ j * L*K + k * L + l  # Valor correlativo
                    self.diccionario_indices[clave] = valor

        max_valor = max(self.diccionario_indices.values())
        for i in range(I):  # i 
            for w in range(W):  # n  
                clave = f"W_{i}_{w}"
                valor = max_valor+1+i*W+w
                self.diccionario_indices[clave] = valor
                
        max_valor = max(self.diccionario_indices.values())
        for i in range(I):  # i 
            for n in range(N):  # n  
                clave = f"N_{i}_{n}"
                valor = max_valor+1+i*N+n
                self.diccionario_indices[clave] = valor


        self.variables_totales= max(self.diccionario_indices.values())
        # Cerramos el archivo.
        f.close()

       

def get_instance_data():
    #file_location = sys.argv[1].strip()
    file_location='data/input_data.txt'
    instance = FieldWorkAssignment()
    instance.load(file_location)
    return instance
     


In [40]:
data=get_instance_data()
data.cantidad_trabajadores
type(data.ordenes)
type(data.ordenes[0])
data.ordenes[0]
data.ordenes[0].id
data.ordenes[0].beneficio
data.diccionario_indices['X_1_5_0_0']
data.diccionario_indices
#data.variables_totales
#data.valor_tramo
#data.ordenes[0].id
#data.ordenes[0].beneficio
#data.ordenes[0].trabajadores_necesarios 
#[0]*(data.cantidad_trabajadores*2)
#data.cantidad_ordenes 
#data.ordenes
#data.conflictos_trabajadores 
#data.ordenes_correlativas 
#data.ordenes_conflictivas
#data.ordenes_repetitivas 
#data.cantidad_ordenes*TURNOS*DIAS

{'X_0_0_0_0': 0,
 'X_0_0_0_1': 1,
 'X_0_0_0_2': 2,
 'X_0_0_0_3': 3,
 'X_0_0_0_4': 4,
 'X_0_0_1_0': 5,
 'X_0_0_1_1': 6,
 'X_0_0_1_2': 7,
 'X_0_0_1_3': 8,
 'X_0_0_1_4': 9,
 'X_0_0_2_0': 10,
 'X_0_0_2_1': 11,
 'X_0_0_2_2': 12,
 'X_0_0_2_3': 13,
 'X_0_0_2_4': 14,
 'X_0_0_3_0': 15,
 'X_0_0_3_1': 16,
 'X_0_0_3_2': 17,
 'X_0_0_3_3': 18,
 'X_0_0_3_4': 19,
 'X_0_0_4_0': 20,
 'X_0_0_4_1': 21,
 'X_0_0_4_2': 22,
 'X_0_0_4_3': 23,
 'X_0_0_4_4': 24,
 'X_0_0_5_0': 25,
 'X_0_0_5_1': 26,
 'X_0_0_5_2': 27,
 'X_0_0_5_3': 28,
 'X_0_0_5_4': 29,
 'X_0_1_0_0': 30,
 'X_0_1_0_1': 31,
 'X_0_1_0_2': 32,
 'X_0_1_0_3': 33,
 'X_0_1_0_4': 34,
 'X_0_1_1_0': 35,
 'X_0_1_1_1': 36,
 'X_0_1_1_2': 37,
 'X_0_1_1_3': 38,
 'X_0_1_1_4': 39,
 'X_0_1_2_0': 40,
 'X_0_1_2_1': 41,
 'X_0_1_2_2': 42,
 'X_0_1_2_3': 43,
 'X_0_1_2_4': 44,
 'X_0_1_3_0': 45,
 'X_0_1_3_1': 46,
 'X_0_1_3_2': 47,
 'X_0_1_3_3': 48,
 'X_0_1_3_4': 49,
 'X_0_1_4_0': 50,
 'X_0_1_4_1': 51,
 'X_0_1_4_2': 52,
 'X_0_1_4_3': 53,
 'X_0_1_4_4': 54,
 'X_0_1_5_0': 55,
 '

In [31]:
# Va or aca


valores_j_0 = []
t=0
for clave, valor in data.diccionario_indices.items():
    partes = clave.split('_')
    if partes[0] == "R" and int(partes[1]) == t:
        valores_j_0.append(valor)

# Imprime la lista de valores
print(valores_j_0)

[3060, 3061, 3062, 3063, 3064, 3065, 3066, 3067, 3068, 3069, 3070, 3071, 3072, 3073, 3074, 3075, 3076, 3077, 3078, 3079, 3080, 3081, 3082, 3083, 3084, 3085, 3086, 3087, 3088, 3089]


In [43]:
data.diccionario_indices['R_0_0_0'],data.diccionario_indices['R_0_0_1'],data.diccionario_indices['R_0_0_2'],data.diccionario_indices['R_0_5_4']

(3060, 3061, 3062, 3089)

In [49]:
print(data.diccionario_indices['N_0_0'],data.diccionario_indices['N_0_1'], data.diccionario_indices['N_0_2'], data.diccionario_indices['N_0_3'])#misma persona 4 tramos

print(data.diccionario_indices['N_1_0'],data.diccionario_indices['N_1_1'], data.diccionario_indices['N_1_2'], data.diccionario_indices['N_1_3'])

3390 3391 3392 3393
3394 3395 3396 3397


In [17]:
lista_beneficio=[]
for i in range(data.cantidad_ordenes):
   lista_beneficio.append(data.ordenes[i].beneficio)

max_valor_posible=sum(lista_beneficio)
lista_beneficio

[5, 6, 5, 7, 5, 5, 5, 5, 5, 10]

In [16]:
data.ordenes[9].beneficio

10

In [32]:
def add_constraint_matrix(my_problem, data):
    
    # Restriccion cada R(j)<=1 la suma de todos los R que sean del mismo j pero distinta k o l deben ser 1

    indices = ...
    values = ...
    row = [indices,values]
    my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[1])

In [69]:
def add_constraint_matrix(my_problem, data):
    
    # Restriccion cada R(j)<=1 la suma de todos los R que sean del mismo j pero distinta k o l deben ser 1
    for j in  range(data.cantidad_ordenes):
        lista= [0]*data.cantidad_ordenes*TURNOS*DIAS
        for i in range((j)*TURNOS*DIAS, (j+1)*TURNOS*DIAS):
            lista[i]=1
        indices = lista+[0]*(data.cantidad_trabajadores*4)     
        values = lista+[0]*(data.cantidad_trabajadores*4)  # porque no esta multiplicado por nada
        row = [indices,values]
        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[1])

In [50]:
def populate_by_row(my_problem, data):
#VER
    # Definimos y agregamos las variables de la parte positiva de la formula beneficio
    coef_fun_objetivo_1 = []
    for i in range(data.cantidad_ordenes):
        valor_orden=data.ordenes[i].beneficio
        for _ in range(data.dias_semana*data.turnos):
            coef_fun_objetivo_1.append(valor_orden)
            
    print(coef_fun_objetivo_1)
    my_problem.variables.add(obj = coef_fun_objetivo_1, lb =[0]*data.cantidad_ordenes*data.dias_semana*data.turnos, ub =[1]*data.cantidad_ordenes*data.dias_semana*data.turnos, types=['B']*data.cantidad_ordenes*data.dias_semana*data.turnos) 
#VER Fin
    # Definimos y agregamos las variables de la parte negativa de la formula costo
    #cada trabajador tiene N1, N2, N3, N4  si el indice que les genera es posterior al resto esta justo!!!
    coef_fun_objetivo_2=data.valor_tramo*data.cantidad_trabajadores

    my_problem.variables.add(obj = coef_fun_objetivo_2, lb =[0]*(data.cantidad_trabajadores*data.tramos_salario), ub =[5]*(data.cantidad_trabajadores*data.tramos_salario), types=['I']*data.cantidad_trabajadores*data.tramos_salario) 

    # Seteamos direccion del problema
    my_problem.objective.set_sense(my_problem.objective.sense.maximize)
    # ~ my_problem.objective.set_sense(my_problem.objective.sense.minimize)

    # Definimos las restricciones del modelo. Encapsulamos esto en una funcion. 
    add_constraint_matrix(my_problem, data)

    # Exportamos el LP cargado en myprob con formato .lp. 
    # Util para debug.
    my_problem.write('balanced_assignment.lp')

In [48]:
coef_fun_objetivo_1 = []
for i in range(data.cantidad_ordenes):
    valor_orden=data.ordenes[i].beneficio
    for _ in range(TURNOS * DIAS):
        coef_fun_objetivo_1.append(valor_orden)
print(coef_fun_objetivo_1)

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [34]:
def solve_lp(my_problem, data):
    
    # Resolvemos el ILP.
    
    my_problem.solve()

    # Obtenemos informacion de la solucion. Esto lo hacemos a traves de 'solution'. 
    x_variables = my_problem.solution.get_values()
    objective_value = my_problem.solution.get_objective_value()
    status = my_problem.solution.get_status()
    status_string = my_problem.solution.get_status_string(status_code = status)

    print('Funcion objetivo: ',objective_value)
    print('Status solucion: ',status_string,'(' + str(status) + ')')

    # Imprimimos las variables usadas.
    for i in range(len(x_variables)):
        # Tomamos esto como valor de tolerancia, por cuestiones numericas.
        if x_variables[i] > TOLERANCE:
            print('x_' + str(data.items[i].index) + ':' , x_variables[i])


In [35]:
data = get_instance_data()

In [61]:
prob_lp = cplex.Cplex()

In [ ]:
populate_by_row(prob_lp,data)

In [ ]:
def main():
    
    # Obtenemos los datos de la instancia.
    data = get_instance_data()
    
    
    
    # Definimos el problema de cplex.
    prob_lp = cplex.Cplex()
    
    # Armamos el modelo.
    populate_by_row(prob_lp,data)

    # Resolvemos el modelo.
    solve_lp(prob_lp,data)


if __name__ == '__main__':
    main()
